# Chapter7 함수 데커레이터와 클로저

## 7.1 데커레이커 기본 지식

In [10]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print('running target()')

target()

running inner()


In [11]:
target

<function __main__.deco.<locals>.inner()>

## 7.2 파이썬이 데커레이터를 실행하는 시점

In [12]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
print('running main()')
print('registry ->', registry)
f1()
f2()
f3()

running register(<function f1 at 0x111ae70d0>)
running register(<function f2 at 0x111ae72f0>)
running main()
registry -> [<function f1 at 0x111ae70d0>, <function f2 at 0x111ae72f0>]
running f1()
running f2()
running f3()


In [14]:
import registration

running register(<function f1 at 0x111ac4c80>)
running register(<function f2 at 0x111ae7730>)


In [16]:
registration.registry

[<function registration.f1()>, <function registration.f2()>]

## 7.3 데커레이터로 개선한 전략 패턴

In [17]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    return order.total() * 0.5 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

## 7.4 변수 범위 규칙

In [18]:
def f1(a):
    print(a)
    print(b)
    
f1(3)

3


NameError: name 'b' is not defined

In [19]:
b = 6
f1(3)

3
6


In [21]:
b = 6 
def f2(a):
    print(a)
    print(b)
    b = 9
    
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [22]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [23]:
b

9

In [24]:
from dis import dis
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [25]:
dis(f2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [26]:
dis(f3)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_GLOBAL             1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## 7.5 클로저

In [32]:
class Averager():
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)
    
avg = Averager()
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

In [33]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    return averager

avg = make_averager()
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

In [35]:
avg.__code__.co_varnames

('new_value', 'total')

In [36]:
avg.__code__.co_freevars

('series',)

In [41]:
avg.__closure__

(<cell at 0x111a54c18: list object at 0x111af9f48>,)

In [48]:
avg.__closure__[0].cell_contents

[10, 11, 12]

## 7.6 nonlocal 선언

In [54]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [56]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    
    return averager

avg = make_averager()
avg(10), avg(11), avg(12)

(10.0, 10.5, 11.0)

## 7.7 간단한 데커레이터 구현하기

In [1]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elpased = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elpased, name, arg_str, result))
        return result
    return clocked

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    ''' factorial'''
    return 1 if n < 2 else n*factorial(n-1)

print('*' * 40, 'Calling snooze(.123)')
snooze(.123)

print('*' * 40, 'Calling factorial(6)')
factorial(6)

**************************************** Calling snooze(.123)
[0.12303862s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000074s] factorial(1) -> 1
[0.00004742s] factorial(2) -> 2
[0.00009787s] factorial(3) -> 6
[0.00013486s] factorial(4) -> 24
[0.00015860s] factorial(5) -> 120
[0.00019855s] factorial(6) -> 720


720

In [69]:
factorial.__name__, factorial.__doc__

('clocked', None)

In [90]:
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args)
        elpased = time.perf_counter() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str=', '.join(arg_lst)
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elpased, name, arg_str, result))
        return result
    return clocked

@clock
def factorial(n):
    ''' factorial n!'''
    return 1 if n < 2 else n*factorial(n-1)

factorial(6)


[0.00000041s] factorial(1) -> 1
[0.00007949s] factorial(2) -> 2
[0.00011074s] factorial(3) -> 6
[0.00013740s] factorial(4) -> 24
[0.00016224s] factorial(5) -> 120
[0.00018545s] factorial(6) -> 720


720

In [91]:
factorial.__name__, factorial.__doc__

('factorial', ' factorial n!')

## 7.8 표준 라이브러리에서 제공하는 데커레이터

In [3]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)
        
print(fibonacci(6))

[0.00000035s] fibonacci(0) -> 0
[0.00000052s] fibonacci(1) -> 1
[0.00010917s] fibonacci(2) -> 1
[0.00000041s] fibonacci(1) -> 1
[0.00000052s] fibonacci(0) -> 0
[0.00000100s] fibonacci(1) -> 1
[0.00009565s] fibonacci(2) -> 1
[0.00031673s] fibonacci(3) -> 2
[0.00050003s] fibonacci(4) -> 3
[0.00000032s] fibonacci(1) -> 1
[0.00000044s] fibonacci(0) -> 0
[0.00000038s] fibonacci(1) -> 1
[0.00013693s] fibonacci(2) -> 1
[0.00020587s] fibonacci(3) -> 2
[0.00000034s] fibonacci(0) -> 0
[0.00000044s] fibonacci(1) -> 1
[0.00004142s] fibonacci(2) -> 1
[0.00000029s] fibonacci(1) -> 1
[0.00000044s] fibonacci(0) -> 0
[0.00000034s] fibonacci(1) -> 1
[0.00004074s] fibonacci(2) -> 1
[0.00007910s] fibonacci(3) -> 2
[0.00017662s] fibonacci(4) -> 3
[0.00042766s] fibonacci(5) -> 5
[0.00097488s] fibonacci(6) -> 8
8


In [7]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

[0.00000065s] fibonacci(0) -> 0
[0.00000064s] fibonacci(1) -> 1
[0.00015301s] fibonacci(2) -> 1
[0.00000108s] fibonacci(3) -> 2
[0.00023230s] fibonacci(4) -> 3
[0.00000093s] fibonacci(5) -> 5
[0.00028547s] fibonacci(6) -> 8
8


In [8]:
fibonacci(30)

[0.00000395s] fibonacci(7) -> 13
[0.00010081s] fibonacci(8) -> 21
[0.00000109s] fibonacci(9) -> 34
[0.00017415s] fibonacci(10) -> 55
[0.00000097s] fibonacci(11) -> 89
[0.00022143s] fibonacci(12) -> 144
[0.00000088s] fibonacci(13) -> 233
[0.00028002s] fibonacci(14) -> 377
[0.00000172s] fibonacci(15) -> 610
[0.00038964s] fibonacci(16) -> 987
[0.00000094s] fibonacci(17) -> 1597
[0.00043635s] fibonacci(18) -> 2584
[0.00000193s] fibonacci(19) -> 4181
[0.00050252s] fibonacci(20) -> 6765
[0.00000091s] fibonacci(21) -> 10946
[0.00061583s] fibonacci(22) -> 17711
[0.00000095s] fibonacci(23) -> 28657
[0.00067632s] fibonacci(24) -> 46368
[0.00000084s] fibonacci(25) -> 75025
[0.00073001s] fibonacci(26) -> 121393
[0.00000082s] fibonacci(27) -> 196418
[0.00078341s] fibonacci(28) -> 317811
[0.00000091s] fibonacci(29) -> 514229
[0.00084198s] fibonacci(30) -> 832040


832040

In [14]:
@functools.lru_cache()
def unhashable(l):
    if len(l) == 0:
        return 1
    return unhashable(l[:-2])

unhashable([1, 2, 3])

TypeError: unhashable type: 'list'

In [17]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [25]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [27]:
htmlize('Heimlich & Co.\n -a game')

'<pre>&#x27;Heimlich &amp; Co.\\n -a game&#x27;</pre>'

In [28]:
htmlize(42)

'<pre>42</pre>'

In [29]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


In [44]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<pre>{}</pre>'.format(content)

@htmlize.register(numbers.Integral)
def _(n):
    return '<p>{0} (0x{0:x})</p>'.format(n)

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [39]:
htmlize('Heimlich & Co.\n -a game')


'<pre>Heimlich &amp; Co.<br>\n -a game</pre>'

In [45]:
htmlize(42)

'<p>42 (0x2a)</p>'

In [46]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<ul>
<li><pre>alpha</pre></li>
<li><p>66 (0x42)</p></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


## 7.10 매개변수화된 데커레이터

In [48]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

print('running main()')
print('register ->', registry)
f1()

running register(<function f1 at 0x103ac6488>)
running main()
register -> [<function f1 at 0x103ac6488>]
running f1()


In [52]:
registry = set()

def register(activate=True):
    def decorate(func):
        print('running register(activate=%s)->decorate(%s)' % (activate, func))
        if activate:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(activate=False)
def f1():
    print('running f1()')
    
@register()
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')
    
registry

running register(activate=False)->decorate(<function f1 at 0x1040b2400>)
running register(activate=True)->decorate(<function f2 at 0x1040b22f0>)


{<function __main__.f2()>}

In [53]:
register()(f3)

running register(activate=True)->decorate(<function f3 at 0x103ac6ae8>)


<function __main__.f3()>

In [55]:
registry

{<function __main__.f2()>, <function __main__.f3()>}

In [56]:
register(activate=False)(f2)

running register(activate=False)->decorate(<function f2 at 0x1040b22f0>)


<function __main__.f2()>

In [57]:
registry

{<function __main__.f3()>}

In [83]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

@clock()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

[0.12314820s] snooze(0.123) -> None
[0.12775707s] snooze(0.123) -> None
[0.12314081s] snooze(0.123) -> None


In [84]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze: 0.12496304512023926s
snooze: 0.12541484832763672s
snooze: 0.1250917911529541s


In [86]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.126s
snooze(0.123) dt=0.124s
snooze(0.123) dt=0.126s
